# Evaluation of Unsupervised Extractive Text Summarization algorithms
## Using word embeddings (BERT vs TFIDF)
- LSA top-n sentences with largest top-n topic weights
- LSA with weighted score
- Textrank
- Matchsum-like (Greedy document similarity maximizer of average document- vs average sentence embeddings)
- K-means of sentence embeddings

## Baselines
- Lead
- Oracle
- Random

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install tensorflow
!pip install tensorflow_datasets 
!pip install spacy
!python -m spacy download en_core_web_lg
!pip install sentence_transformers
!pip install rouge-score
!pip install datasets

In [4]:

from sklearn.feature_extraction.text import TfidfVectorizer
import spacy 
import numpy as np
from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset
import scipy
from rouge_score import rouge_scorer
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import numpy as np
import random

n_sentences = 2

sbert = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
nlp = spacy.load("en_core_web_lg")
tfidf_vectorizer = TfidfVectorizer(stop_words="english")


txt = """ This happened when we first started dating.
His mobile used to ping and light up when someone texted him and once I saw that someone had texted him with the name "Babe ❤️" I didn't think of it as anything and that I had not seen it correctly but next coming weeks I saw it happening many times. He would get really happy when the person texted, used to smile really big and all. I started thinking I was being cheated on, the last straw for me was when the person texted "love you too" I confronted him about and he stared at me for some time before he started laughing.
I cried because what the hell, so he calmed me down and explained to me that Babe is his grandmother, her name is Baberuth and everyone in the family called her Babe, she recently had gotten her first smartphone and he had taught her to text, so when she texted it was exciting for him to see her using emojis and stuff.
Never felt so embarrassed in my life. A few months later he took me to meet her and I kid you not, for a 85 year old Babe is a sport. Seeing her punch a guy square in the jaw for harassing was a sight. We are best friends now. """

summary = """I assumed my boyfriend was cheating on me but it was just his grandmother texting him."""

# Baselines and Metrics
- Getting sentences
- Rouge scores
- Oracle
- Lead
- Random Summary


In [5]:

def get_sentences(txt):
  if isinstance(txt, list):
    return(txt)
  
  doc = nlp(txt) 
  sentences = []
  for sent in doc.sents:
      sentences.append(" ".join([token.text for token in sent]))
  return(sentences)

def rouge_scores(pred, summary, scores = ['rouge1', 'rouge2', 'rougeL'], scorer = None):
  if scorer == None:
    scorer = rouge_scorer.RougeScorer(scores, use_stemmer=True)
  scores = scorer.score(pred, summary)
  return [float("{:.4f}".format(s.fmeasure)) for k, s in scores.items()]

def oracle(txt, summary, maximize_score = 'rouge2'):
  assert maximize_score in ['rouge1', 'rouge2', 'rougeL']
  def get_summary(sentence_idx_list):
      sorted_summary = sorted(sentence_idx_list, key = lambda x : x[0])
      current_summary = ". ".join([entry[1] for entry in sorted_summary])
      return(current_summary)

  sentences = get_sentences(txt)
  best_sentences = []
  best_summary = ""
  if n_sentences >= len(sentences):
    return(" ".join(sentences))
  while len(best_sentences) < n_sentences:
    scores = []
    for idx, sent in enumerate(sentences):
      current_summary = get_summary(best_sentences + [[idx, sent]])
      scores.append(rouge_scores(current_summary, summary, [maximize_score])[0])
    best_s_idx = np.argmax(scores)
    best_sentences.append([best_s_idx, sentences.pop(best_s_idx)])
  return(get_summary(best_sentences))


def lead(txt):
  sentences = get_sentences(txt)
  if n_sentences>=len(sentences):
    top_sentences = sentences
  else:
    top_sentences = sentences[0:n_sentences]

  return(". ".join(top_sentences))

def rand(txt):
  sentences = get_sentences(txt)
  if n_sentences>=len(sentences):
    top_sentences = sentences
  else:
    indexes = random.sample(range(0, len(sentences)), n_sentences)
    indexes.sort()
    top_sentences = [sentences[ind] for ind in indexes]
  return(". ".join(top_sentences))


# LSA Summary
- Non-Weighted
- Weighted

In [6]:

def LSA_summary(txt, embedding_function):  
  sentences = get_sentences(txt)
  if n_sentences>=len(sentences):
    top_sentences = sentences
  else:
    # Word x Sentence Matrix
    sentence_embedding_matrix = embedding_function(sentences)
    k = min(max(6,n_sentences),min(sentence_embedding_matrix.shape))
    U, sigma, V = scipy.sparse.linalg.svds(sentence_embedding_matrix, which="LM", k=k)
    top_sentences = []
    ind2ind = [i for i in range(len(sentences))]
    for i in range(1,1+n_sentences):
      best_sentence = np.argmax(V[-i,:])
      sentence_idx = ind2ind[best_sentence]
      top_sentences.append([sentence_idx, sentences[sentence_idx]])
      # remove the sentence column and adjust indexes
      V = np.delete(V, best_sentence, axis=1)
      ind2ind.pop(best_sentence)
    top_sentences.sort(key =  lambda x: x[0])
    top_sentences = [x[1] for x in top_sentences]

  return(". ".join(top_sentences))

def LSA_summary_weighted(txt, embedding_function):
  sentences = get_sentences(txt)
  if n_sentences>=len(sentences):
    top_sentences = sentences
  else:
    # Word x Sentence Matrix
    sentence_embedding_matrix = embedding_function(sentences)
    k = min(max(6,n_sentences),min(sentence_embedding_matrix.shape))
    U, sigma, V = scipy.sparse.linalg.svds(sentence_embedding_matrix, which="LM", k=k)
    sentence_scores = np.sqrt(np.square(sigma) @ np.square(V))
    sentence_scores_idx = [[i,sentence_scores[i]] for i in range(len(sentence_scores))]
    sentence_scores_idx.sort(key = lambda x:x[1])
    top_sentences_idx_score = sentence_scores_idx[-1:-(n_sentences+1):-1]
    top_sentences_idx_score.sort(key =  lambda x: x[0])
    top_sentences = [sentences[x[0]] for x in top_sentences_idx_score]

  return(". ".join(top_sentences))

def LSA_TFIDF(txt, weighted = False):
  embedding_function = lambda sentences: np.transpose(tfidf_vectorizer.fit_transform(sentences))
  if weighted:
    return(LSA_summary_weighted(txt, embedding_function))
  else:
    return(LSA_summary(txt, embedding_function))

def LSA_BERT(txt, weighted = False):
  embedding_function = lambda sentences: np.transpose(np.array([sbert.encode(sentence, convert_to_numpy=True) for sentence in sentences]))
  if weighted:
    return(LSA_summary_weighted(txt, embedding_function))
  else:
    return(LSA_summary(txt, embedding_function))



weighted = True
lead_summary = lead(txt)
oracle_summary = oracle(txt, summary)
lsa_tfidf_summary = LSA_TFIDF(txt, weighted=weighted)
lsa_bert_summary = LSA_BERT(txt, weighted=weighted)

lead_score = rouge_scores(lead_summary, summary)
oracle_score = rouge_scores(oracle_summary, summary)
lsa_tfidf_scores = rouge_scores(lsa_tfidf_summary, summary)
lsa_bert_score = rouge_scores(lsa_bert_summary, summary)

print(f"LEAD SUMMARY: Score:{lead_score} txt:{lead_summary}")
print(f"ORACLE SUMMARY: Score:{oracle_score} txt:{oracle_summary}")
print(f"TFIDF SUMMARY: Score:{lsa_tfidf_scores} txt:{lsa_tfidf_summary}")
print(f"BERT SUMMARY: Score:{lsa_bert_score} txt:{lsa_bert_summary}")
print(f"REAL SUMMARY: {summary}")

LEAD SUMMARY: Score:[0.1714, 0.0606, 0.1714] txt:  This happened when we first started dating . 
. His mobile used to ping and light up when someone texted him
ORACLE SUMMARY: Score:[0.25, 0.087, 0.1667] txt:His mobile used to ping and light up when someone texted him. I cried because what the hell , so he calmed me down and explained to me that Babe is his grandmother
TFIDF SUMMARY: Score:[0.2069, 0.0, 0.1379] txt:I saw it happening many times .. Never felt so embarrassed in my life .
BERT SUMMARY: Score:[0.1481, 0.0, 0.1481] txt:I saw it happening many times .. We are best friends now .
REAL SUMMARY: I assumed my boyfriend was cheating on me but it was just his grandmother texting him.


# TextRank 

In [7]:

def pagerank(M, num_iterations: int = 100, d: float = 0.85):

    # Row normalization
    #sum_of_rows = M.sum(axis=1)
    #M = M / sum_of_rows[:, np.newaxis]
    N = M.shape[1]
    v = np.random.rand(N, 1)
    v = v / np.linalg.norm(v, 1)
    M_hat = (d * M + (1 - d) / N)
    for i in range(num_iterations):
        v = M_hat @ v
    return v

def textrank_summary(txt, sentence_similarity, embedding_function):

  sentences = get_sentences(txt)
  encoded_sentences = embedding_function(sentences)
  n = len(sentences)
  similarity_matrix = np.zeros((n, n))
  for idx1 in range(n):
        for idx2 in range(n):
            if not idx1==idx2:
              similarity_matrix[idx1][idx2] = sentence_similarity(encoded_sentences[idx1], encoded_sentences[idx2])
  scores = pagerank(similarity_matrix, 100, 0.85)

  # Sort over textrank scores
  ranked_sentences = sorted(((scores[i],i,s) for i,s in enumerate(sentences)), reverse=True)    
  top_sentences = ranked_sentences[0:n_sentences]
  # Sort over ordering for readability
  top_sentences.sort(key = lambda x:x[1])

  summary = ". ".join([s[2] for s in top_sentences])
  return(summary)

def textrank_BERT(txt):
    sentence_similarity = lambda s1, s2: util.pytorch_cos_sim(s1, s2)
    embedding_function = lambda sentences: [sbert.encode(s, convert_to_tensor=True) for s in sentences]
    return(textrank_summary(txt, sentence_similarity, embedding_function))

def textrank_TFIDF(txt):
    sentence_similarity = lambda s1, s2: 1-scipy.spatial.distance.cosine(s1,s2)
    embedding_function = lambda sentences: tfidf_vectorizer.fit_transform(sentences).toarray()
    return(textrank_summary(txt, sentence_similarity, embedding_function))

textrank_tfidf_summary = textrank_TFIDF(txt)
textrank_bert_summary = textrank_BERT(txt)
textrank_tfidf_scores = rouge_scores(textrank_tfidf_summary, summary)
textrank_bert_score = rouge_scores(textrank_bert_summary, summary)

print(f"TFIDF SUMMARY: Score:{textrank_tfidf_scores} txt:{textrank_tfidf_summary}")
print(f"BERT SUMMARY: Score:{textrank_bert_score} txt:{textrank_bert_summary}")
print(f"REAL SUMMARY: {summary}")


TFIDF SUMMARY: Score:[0.32, 0.0833, 0.28] txt:and once I saw that someone had texted him with the name " Babe ❤ ️. I started thinking I was being cheated on , the last straw for me was when the person texted " love you too "
BERT SUMMARY: Score:[0.2642, 0.0392, 0.2642] txt:He would get really happy when the person texted , used to smile really big and all .. I started thinking I was being cheated on , the last straw for me was when the person texted " love you too "
REAL SUMMARY: I assumed my boyfriend was cheating on me but it was just his grandmother texting him.


# Matchsum-U

In [10]:
# compare average sentence embedding to document sentence embedding and pick iteratively sentences that moves closest.
def matchsum(txt, similarity_function, embedding_function):
  sentences = get_sentences(txt)
  
  sentence_embeddings = embedding_function(sentences)
  document_embedding = sum(sentence_embeddings)/len(sentence_embeddings)
  best_sentences = []
  while len(best_sentences)<n_sentences:
    similarities = []
    if best_sentences == []:
      current_embeddings = []
    else:
      current_embeddings = [s[2] for s in best_sentences]
    for emb in sentence_embeddings:
      potential_embedding = sum([emb] + current_embeddings)/(len(current_embeddings)+1)
      similarities.append(similarity_function(potential_embedding, document_embedding))
    best_sentence_idx = np.argmax(similarities)
    best_sentences.append([best_sentence_idx,sentences[best_sentence_idx],sentence_embeddings[best_sentence_idx]])
  best_sentences.sort(key=lambda x:x[0])
  summary = ". ".join([s[1] for s in best_sentences])
  return(summary)


def matchsum_BERT(txt):
  similarity_function = lambda s1, s2: 1-scipy.spatial.distance.cosine(s1,s2)
  embedding_function = lambda sentences: [sbert.encode(s, convert_to_numpy=True) for s in sentences]
  return(matchsum(txt, similarity_function, embedding_function))

def matchsum_TFIDF(txt):
  similarity_function = lambda s1, s2: 1-scipy.spatial.distance.cosine(s1,s2)
  embedding_function = lambda sentences: tfidf_vectorizer.fit_transform(sentences).toarray()
  return(matchsum(txt, similarity_function, embedding_function))
        

matchsum_summary_bert = matchsum_BERT(txt)
matchsum_summary_tfidf = matchsum_TFIDF(txt)
matchsum_tfidf_scores = rouge_scores(matchsum_summary_tfidf, summary)
matchsum_bert_score = rouge_scores(matchsum_summary_bert, summary)

print(f"TFIDF SUMMARY: Score:{matchsum_tfidf_scores} txt:{matchsum_summary_tfidf}")
print(f"BERT SUMMARY: Score:{matchsum_bert_score} txt:{matchsum_summary_bert}")
print(f"REAL SUMMARY: {summary}")
  

TFIDF SUMMARY: Score:[0.32, 0.0833, 0.28] txt:and once I saw that someone had texted him with the name " Babe ❤ ️. I started thinking I was being cheated on , the last straw for me was when the person texted " love you too "
BERT SUMMARY: Score:[0.3721, 0.0488, 0.3256] txt:I saw it happening many times .. I started thinking I was being cheated on , the last straw for me was when the person texted " love you too "
REAL SUMMARY: I assumed my boyfriend was cheating on me but it was just his grandmother texting him.


# Clustering

In [13]:

def cluster_summary(txt, similarity_function, embedding_function):
  sentences = get_sentences(txt)
  encoded = embedding_function(sentences)

  kmeans = KMeans(n_clusters=n_sentences)
  kmeans = kmeans.fit(encoded)
  avg = []
  for j in range(n_sentences):
      idx = np.where(kmeans.labels_ == j)[0]
      avg.append(np.mean(idx))
  closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, encoded)
  ordering = sorted(range(n_sentences), key=lambda k: avg[k])
  summary = '. '.join([sentences[closest[idx]] for idx in ordering])
  return(summary)

def cluster_TFIDF(txt):
  similarity_function = lambda s1, s2: 1-scipy.spatial.distance.cosine(s1,s2)
  embedding_function = lambda sentences: tfidf_vectorizer.fit_transform(sentences).toarray()
  return(cluster_summary(txt, similarity_function, embedding_function))

def cluster_BERT(txt):
  similarity_function = lambda s1, s2: 1-scipy.spatial.distance.cosine(s1,s2)
  embedding_function = lambda sentences: [sbert.encode(s, convert_to_numpy=True) for s in sentences]
  return(cluster_summary(txt, similarity_function, embedding_function))

cluster_summary_bert = cluster_BERT(txt)
cluster_summary_tfidf = cluster_TFIDF(txt)
cluster_tfidf_scores = rouge_scores(cluster_summary_tfidf, summary)
cluster_bert_score = rouge_scores(cluster_summary_bert, summary)

print(f"TFIDF SUMMARY: Score:{cluster_tfidf_scores} txt:{cluster_summary_tfidf}")
print(f"BERT SUMMARY: Score:{cluster_bert_score} txt:{cluster_summary_bert}")
print(f"REAL SUMMARY: {summary}")


TFIDF SUMMARY: Score:[0.32, 0.0833, 0.32] txt:I started thinking I was being cheated on , the last straw for me was when the person texted " love you too ". and once I saw that someone had texted him with the name " Babe ❤ ️
BERT SUMMARY: Score:[0.2979, 0.0444, 0.2979] txt:I started thinking I was being cheated on , the last straw for me was when the person texted " love you too ". A few months later he took me to meet her
REAL SUMMARY: I assumed my boyfriend was cheating on me but it was just his grandmother texting him.


# Saving results to G-Drive

In [11]:
import datetime
import json
from google.colab import drive
drive.mount('/content/gdrive')

def save_results_gdrive(score_data):
  current_time = datetime.datetime.now()  
  file_name = f"Results{current_time}"

  score_avg = score_data.copy()
  for key in score_avg.keys():
    if isinstance(score_avg[key], list):
      score_avg[key] = np.mean(np.array(score_avg[key]), axis=0)
      score_avg[key].tolist()

  class NumpyEncoder(json.JSONEncoder):
      """ Special json encoder for numpy types """
      def default(self, obj):
          if isinstance(obj, np.integer):
              return int(obj)
          elif isinstance(obj, np.floating):
              return float(obj)
          elif isinstance(obj, np.ndarray):
              return obj.tolist()
          return json.JSONEncoder.default(self, obj)
  print(score_avg)
  dumped = json.dumps(score_avg, cls=NumpyEncoder)

  with open(f'/content/gdrive/MyDrive/Colab Notebooks/{file_name}.json', 'w') as fp:
      json.dump(dumped, fp)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Test Script

In [ ]:
from tqdm import tqdm
import warnings

warnings.filterwarnings(action='once')
save_interval = 50
data_name = 'cnn_dailymail'
dataset = load_dataset(data_name, '3.0.0')
test_data = dataset['test']
weighted = False

score_data = {'dataset': data_name, 'n_sentences':n_sentences,
             'weighted_lsa': weighted,
              'tfidf_cluster':[], 'bert_cluster':[], 
              'tfidf_matchsum':[], 'bert_matchsum':[],
              'tfidf_textrank':[], 'bert_textrank':[],
              'tfidf_lsa':[], 'bert_lsa':[],
              'oracle':[], 'lead':[], 'random':[]}

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def test_baselines(sentences, gold_summary, scorer = None):
  lead_summary = lead(sentences)
  oracle_summary = oracle(sentences, gold_summary)
  rand_summary = rand(sentences)
  lead_score = rouge_scores(lead_summary, gold_summary, scorer = scorer)
  oracle_score = rouge_scores(oracle_summary, gold_summary, scorer = scorer)
  rand_score = rouge_scores(rand_summary, gold_summary)
  score_data['oracle'].append(oracle_score)
  score_data['lead'].append(lead_score)
  score_data['random'].append(rand_score)

def test_lsa(sentences, gold_summary, scorer = None):
  lsa_tfidf_summary = LSA_TFIDF(sentences, weighted=weighted)
  lsa_bert_summary = LSA_BERT(sentences, weighted=weighted)
  lsa_tfidf_scores = rouge_scores(lsa_tfidf_summary, gold_summary, scorer = scorer)
  lsa_bert_score = rouge_scores(lsa_bert_summary, gold_summary, scorer = scorer)
  score_data['tfidf_lsa'].append(lsa_tfidf_scores)
  score_data['bert_lsa'].append(lsa_bert_score)

def test_matchsum(sentences, gold_summary, scorer = None):
  matchsum_summary_bert = matchsum_BERT(sentences)
  matchsum_summary_tfidf = matchsum_TFIDF(sentences)
  matchsum_tfidf_scores = rouge_scores(matchsum_summary_tfidf, gold_summary, scorer = scorer)
  matchsum_bert_score = rouge_scores(matchsum_summary_bert, gold_summary, scorer = scorer)
  score_data['tfidf_matchsum'].append(matchsum_tfidf_scores)
  score_data['bert_matchsum'].append(matchsum_bert_score)

def test_textrank(sentences, gold_summary, scorer = None):
  textrank_tfidf_summary = textrank_TFIDF(sentences)
  textrank_bert_summary = textrank_BERT(sentences)
  textrank_tfidf_scores = rouge_scores(textrank_tfidf_summary, gold_summary, scorer = scorer)
  textrank_bert_score = rouge_scores(textrank_bert_summary, gold_summary, scorer = scorer)
  score_data['tfidf_textrank'].append(textrank_tfidf_scores)
  score_data['bert_textrank'].append(textrank_bert_score)


def test_cluster(sentences, gold_summary, scorer = None):
  cluster_summary_bert = cluster_BERT(sentences)
  cluster_summary_tfidf = cluster_TFIDF(sentences)
  cluster_tfidf_scores = rouge_scores(cluster_summary_tfidf, gold_summary, scorer = scorer)
  cluster_bert_score = rouge_scores(cluster_summary_bert, gold_summary, scorer = scorer)
  score_data['tfidf_cluster'].append(cluster_tfidf_scores)
  score_data['bert_cluster'].append(cluster_bert_score)


for i in tqdm(range(len(test_data))):
  text = test_data[i]['article']
  gold_summary = test_data[i]['highlights']
  sentences = get_sentences(text)
  if len(sentences) > 60:
    continue

  test_functions = [test_baselines, test_lsa, test_matchsum, test_textrank, test_cluster]
  for summary_task in test_functions:
    try:
      summary_task(sentences, gold_summary, scorer)
    except ValueError as e:
      print(e)
  if i % save_interval == 0:
    save_results_gdrive(score_data)


Display previous results

In [16]:
file_name = 'At-Home-Results-1-Weighted'
with open(f'/content/gdrive/MyDrive/Colab Notebooks/{file_name}.json', 'r') as fp:
    print(json.load(fp))


{"dataset": "cnn_dailymail", "n_sentences": 1, "weighted_lsa": true, "tfidf_lsa": [0.18607097334878334, 0.061866888760139155, 0.13962882387022058], "bert_lsa": [0.08258093858632673, 0.020380272305909584, 0.06550648899188838]}


Sources used:

[Non-Deep Learning methods](https://medium.com/jatana/unsupervised-text-summarization-using-sentence-embeddings-adb15ce83db1)

[List of SOTA](http://nlpprogress.com/english/summarization.html)

[Extractive text with BERT (MATCHSUM) ](https://arxiv.org/pdf/2004.08795.pdf)

[LSA](https://www.researchgate.net/publication/220195824_Text_summarization_using_Latent_Semantic_Analysis)


[ROUGE](https://www.aclweb.org/anthology/W04-1013.pdf)

[Textrank](https://towardsdatascience.com/understand-text-summarization-and-create-your-own-summarizer-in-python-b26a9f09fc70)

[Oracle (under extractive training)](https://arxiv.org/pdf/1611.04230.pdf
)


[TextRank](https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf
)

